### <font color='lightblue'>Análisis Equipos Premier League temporadas 2022 - 2023  &  2023 - 2024</font>  

<font color='lightblue'>Importe librerias a utilizar</font>  

In [219]:
import pandas as pd
import numpy as np
import xlwings as xw
import plotly.express as px
import matplotlib as plt
from scipy.stats import poisson
import matplotlib.pyplot as plt

In [14]:
# import module
import openpyxl

<font color='lightblue'>Lectura y unificación de datos</font>  

In [2]:
new_df=pd.read_csv('2223.csv')  

df_2_temporada_actual=pd.read_csv('2324.csv')  

In [3]:
df= pd.concat([new_df,df_2_temporada_actual], ignore_index=True)


In [4]:
# Environment settings: 
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)


<font color='lightblue'>Links de datos y diccionario de variables</font>  



https://www.football-data.co.uk/notes.txt

https://www.football-data.co.uk/englandm.php


### <font color='lightblue'>Análisis general equipos premier league como local</font>  

In [5]:
def estadisticas_equipos_local(data):
    a=data.groupby(["HomeTeam"])[["FTHG","HS","HST","HC","HY","HTHG"]].agg("mean").reset_index().sort_values(by="FTHG", ascending=False)
    return a

def rendimiento_equipos_local(data):
    unique_equipo = data["HomeTeam"].unique()
    lista=list()
    for i in unique_equipo:
        lista.append(data[data["HomeTeam"]==i][["HomeTeam","FTR"]].value_counts().to_frame().reset_index())

    dfr = pd.concat(lista)
    return dfr
   

### <font color='lightblue'>Análisis general equipos premier league como visitante</font>  

In [6]:
def estadisticas_equipos_visita(data):
    a=data.groupby(["AwayTeam"])[["FTAG","AS","AST","AC","AY","HTAG"]].agg("mean").reset_index().sort_values(by="FTAG", ascending=False)
    return a

def rendimiento_equipos_visita(data):
    unique_equipo = data["AwayTeam"].unique()
    lista=list()
    for i in unique_equipo:
        lista.append(data[data["AwayTeam"]==i][["AwayTeam","FTR"]].value_counts().to_frame().reset_index())

    dfr = pd.concat(lista)
    return dfr

### <font color='lightblue'>Análisis general equipos premier league, ultimos 5 partidos como local</font>  

In [309]:
def estadisticas_equipos_local_ultimos_5(data):
    unique_equipo = data["HomeTeam"].unique()
    lista=list()

    for i in unique_equipo:
        a=data[data["HomeTeam"]==i].sort_values(by='Date', ascending=False).head(5).groupby(["HomeTeam"])[["FTHG","HS","HST","HC","HY","HTHG"]].agg("mean").reset_index()
        lista.append(a)
    dfr = pd.concat(lista).sort_values(by='FTHG', ascending=False)
    return dfr


def rendimiento_equipos_local_ultimos_5(data, Equipo):
   
    dfr=data[data["HomeTeam"]==Equipo].sort_values(by='Date', ascending=False).head(5)[["HomeTeam","FTR"]].value_counts().to_frame().reset_index()

    return dfr



def probas_local(data, Equipo):
    datos_equipo=data[data["HomeTeam"]==Equipo].sort_values(by='Date', ascending=False).reset_index()
    media=datos_equipo["FTHG"].mean()
    lista_k=[0,1,2,3,4,5,6,7]
    goles=["0 Goles", "1 Goles", "2 Goles", "3 Goles","4 Goles", "5 Goles", "6 Goles", "7 Goles"]
    lista_probabilidad=[]
    for i in lista_k:
        lista_probabilidad.append(poisson.pmf(i, mu=media))
    df_listo=pd.DataFrame(list(zip(goles, lista_probabilidad)), columns=["Goles","Probabilidad"])
    return df_listo



def probas_acumuladas_local(data, Equipo):
    datos_equipo=data[data["HomeTeam"]==Equipo].sort_values(by='Date', ascending=False).reset_index()
    media=datos_equipo["FTHG"].mean()
    lista_k=[0,1,2,3,4,5,6,7]   
    goles=["Mas de 0", "Mas de 1", "Mas de 2", "Mas de 3","Mas de 4", "Mas de 5", "Mas de 6", "Mas de 7"]
    lista_probabilidad=[]
    for i in lista_k:
        lista_probabilidad.append(1-poisson.cdf(k=i, mu=media))
    
    df_listo_2=pd.DataFrame(list(zip(goles, lista_probabilidad)), columns=["Goles","Probabilidad"])
    return df_listo_2


def probas_acumuladas_local_2(data, Equipo):
    
    
    datos_equipo=data[data["HomeTeam"]==Equipo].sort_values(by='Date', ascending=False).reset_index()
    media=datos_equipo["FTHG"].mean()
    lista_k=[0,1,2,3,4,5,6,7]   
    goles=["<= a 0", "<= a 1", "<= a 2", "<= a 3","<= a 4", "<= a 5", "<= a 6", "<= a 7"]
    lista_probabilidad=[]
    for i in lista_k:
        lista_probabilidad.append(poisson.cdf(k=i, mu=media))
    
    df_listo_2=pd.DataFrame(list(zip(goles, lista_probabilidad)), columns=["Goles","Probabilidad"])
    return df_listo_2



#scando grafico histograma de los goles del local historico
def histograma_goles_local(data, Equipo):
    datos_equipo=data[data["HomeTeam"]==Equipo].sort_values(by='Date', ascending=False).reset_index()
    fig = plt.figure(figsize=(4,2))
    plt.hist(datos_equipo["FTHG"], density=False, label=' Histograma Goles local', color = "seagreen")

    return fig 

### <font color='lightblue'>Análisis general equipos premier league, ultimos 5 partidos como visitante</font>  

In [312]:
def estadisticas_equipos_visita_ultimos_5(data):
    unique_equipo = data["AwayTeam"].unique()
    lista=list()

    for i in unique_equipo:
        a=data[data["AwayTeam"]==i].sort_values(by='Date', ascending=False).head(5).groupby(["AwayTeam"])[["FTAG","AS","AST","AC","AY","HTAG"]].agg("mean").reset_index()
        lista.append(a)
    dfr = pd.concat(lista).sort_values(by='FTAG', ascending=False)
    return dfr


def rendimiento_equipos_visita_ultimos_5(data, Equipo):
  
    dfr=data[data["AwayTeam"]==Equipo].sort_values(by='Date', ascending=False).head(5)[["AwayTeam","FTR"]].value_counts().to_frame().reset_index()

    return dfr


def probas_visita(data, Equipo):
    datos_equipo=data[data["AwayTeam"]==Equipo].sort_values(by='Date', ascending=False).reset_index()
    media=datos_equipo["FTAG"].mean()
    lista_k=[0,1,2,3,4,5,6,7]
    goles=["0 Goles", "1 Goles", "2 Goles", "3 Goles","4 Goles", "5 Goles", "6 Goles", "7 Goles"]
    lista_probabilidad=[]
    for i in lista_k:
        lista_probabilidad.append(poisson.pmf(i, mu=media))
    df_listo=pd.DataFrame(list(zip(goles, lista_probabilidad)), columns=["Goles","Probabilidad"])
    return df_listo



def probas_acumuladas_visita(data, Equipo):
    datos_equipo=data[data["AwayTeam"]==Equipo].sort_values(by='Date', ascending=False).reset_index()
    media=datos_equipo["FTAG"].mean()
    
    lista_k=[0,1,2,3,4,5,6,7]
    goles=["Mas de 0", "Mas de 1", "Mas de 2", "Mas de 3","Mas de 4", "Mas de 5", "Mas de 6", "Mas de 7"]
    lista_probabilidad=[]
    for i in lista_k:
        lista_probabilidad.append(1-poisson.cdf(k=i, mu=media))
    
    df_listo_2=pd.DataFrame(list(zip(goles, lista_probabilidad)), columns=["Goles","Probabilidad"])
    return df_listo_2

def probas_acumuladas_visita_2(data, Equipo):
    
    
    datos_equipo=data[data["AwayTeam"]==Equipo].sort_values(by='Date', ascending=False).reset_index()
    media=datos_equipo["FTAG"].mean()
    lista_k=[0,1,2,3,4,5,6,7]   
    goles=["<= a 0", "<= a 1", "<= a 2", "<= a 3","<= a 4", "<= a 5", "<= a 6", "<= a 7"]
    lista_probabilidad=[]
    for i in lista_k:
        lista_probabilidad.append(poisson.cdf(k=i, mu=media))
    
    df_listo_2=pd.DataFrame(list(zip(goles, lista_probabilidad)), columns=["Goles","Probabilidad"])
    return df_listo_2



#scando grafico histograma de los goles del visita historico
def histograma_goles_visita(data, Equipo):
    datos_equipo=data[data["AwayTeam"]==Equipo].sort_values(by='Date', ascending=False).reset_index()
    fig = plt.figure(figsize=(4,2))
    plt.hist(datos_equipo["FTAG"], density=False, label=' Histograma Goles visita', color = "orange")

    return fig 


    

### <font color='lightblue'>Reporte de la Premier League 2022 - 2023 & 2023 - 2024</font>  

In [326]:
def reporte_apuestas_premier(data, local, visita):
    # Open up a new Excel Workbook
    wb = xw.Book()
    wb.save("Reporte.xlsx")
    ws2=wb.sheets.add("Analisis")
    ws2['A4'].value = estadisticas_equipos_local_ultimos_5(data)
    # Delete Column A
    ws2.range('A:A').api.Delete()
    #adding color
    ws2.range("A4:G4").color = (0,128,128)
    ws2.range("A5:G26").color = (204,255,255)

    ws2['J4'].value = estadisticas_equipos_visita_ultimos_5(data)
    ws2.range('J:J').api.Delete()
    ws2.range("J4:P4").color = (0,128,128)
    ws2.range("J5:P26").color = (204,255,255)

    ws2.range('B1').value="Promedio estadísticas equipo local ultimos 5 partidos"
    ws2.range('B1').font.bold = True
    ws2.range('B1').font.size=14

    ws2.range('K1').value="Promedio estadísticas equipo visita ultimos 5 partidos"
    ws2.range('K1').font.bold = True
    ws2.range('K1').font.size=14

    ws2.range('A30').value=rendimiento_equipos_local_ultimos_5(data, local)
   
    ws2.range("B30:D30").color = (51,153,102)
    
    
    ws2.range('J30').value=rendimiento_equipos_visita_ultimos_5(data, visita)
    ws2.range("K30:M30").color = (255,153,0)
    

    ws2.range('B28').value="Rendimiento local ultimos 5 partidos"
    ws2.range("B28").font.bold = True
    ws2.range('B28').font.size=14

    ws2.range('K28').value="Rendimiento visita ultimos 5 partidos"
    ws2.range("K28").font.bold = True
    ws2.range('K28').font.size=14


    for i in ws2.range("A5:A30"):
        if i.value==local:
            ws2.range(i.address).color=(51,153,102)
            
            ws2.range(ws2.range(i.address).expand("right").address).color=(51,153,102)
    
    
    for i in ws2.range("J5:J30"):
        if i.value==visita:
            ws2.range(i.address).color=(255,153,0)
            
            ws2.range(ws2.range(i.address).expand("right").address).color=(255,153,0) 


    ws3=wb.sheets.add("Probabilidades")
    ws3['A4'].value = probas_local(data, local)
    ws3.range('A:A').api.Delete()
    ws3.range("A4:B4").color = (51,153,102)
    ws3.range("A5:B12").color = (204,255,255)

    ws3.range('A2').value=f"Probabilidad que el local {local} marque x goles"
    ws3.range('A2').font.bold = True
    ws3.range('A2').font.size=11
    
    ws3['E4'].value = probas_acumuladas_local(data, local)
    ws3.range('E:E').api.Delete()
    ws3.range("E4:F4").color = (51,153,102)
    ws3.range("E5:F12").color = (204,255,255)

    ws3.range('E2').value=f"Probabilidad que el local {local} marque más de x goles"
    ws3.range('E2').font.bold = True
    ws3.range('E2').font.size=11


    ws3['A16'].value = probas_visita(data, visita)
    ws3.range('A16:A28').api.Delete()
    ws3.range("A16:B16").color = (255,153,0)
    ws3.range("A17:B24").color = (204,255,255)
    ws3.range('A14').value=f"Probabilidad que el visita {visita} marque x goles"
    ws3.range('A14').font.bold = True
    ws3.range('A14').font.size=11


    ws3['E16'].value = probas_acumuladas_visita(data, visita)
    ws3.range('E16:E28').api.Delete()
    ws3.range("E16:F16").color = (255,153,0)
    ws3.range("E17:F24").color = (204,255,255)
    ws3.range('E14').value=f"Probabilidad que el visita {visita} marque más de x goles"
    ws3.range('E14').font.bold = True
    ws3.range('E14').font.size=11



    ws3.pictures.add(histograma_goles_local(data, local), name='MyPlot', update=True, left=ws3.range('J3').left, top=ws2.range('J3').top)
    ws3.range('J2').value=f"Histograma Goles equipo local {local}"
    ws3.range('J2').font.bold = True
    ws3.range('J2').font.size=11



    ws3.pictures.add(histograma_goles_visita(data, visita), name='MyPlot_2', update=True, left=ws3.range('J16').left, top=ws2.range('J16').top)
    ws3.range('J14').value=f"Histograma Goles equipo visita {visita}"
    ws3.range('J14').font.bold = True
    ws3.range('J14').font.size=11



    ws3['O4'].value = probas_acumuladas_local_2(data, local)
    ws3.range('O:O').api.Delete()
    ws3.range("O4:P4").color = (51,153,102)
    ws3.range("O5:P12").color = (204,255,255)
    ws3.range('O2').value=f"Probabilidad que el local {local} marque igual o menos que x goles"
    ws3.range('O2').font.bold = True
    ws3.range('O2').font.size=11


    
    ws3['O16'].value = probas_acumuladas_visita_2(data, visita)
    ws3.range('O16:O30').api.Delete()
    ws3.range("O16:P16").color = (255,153,0)
    ws3.range("O17:P24").color = (204,255,255)
    ws3.range('O14').value=f"Probabilidad que el visita {visita} marque igual o menos que x goles"
    ws3.range("O14").font.bold = True
    ws3.range('O14').font.size=11


  
    ws3.range("A2:S2").color = (51,153,102)
    ws3.range("A14:S14").color = (255,153,0)
   

In [329]:
reporte_apuestas_premier(df,"Chelsea","Arsenal")